In [1]:
import asyncio
# import psycopg
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker, AsyncSession
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import text, create_engine
from config import settings

In [2]:
# Настройка синхронного двигателя
sync_engine = create_engine(
    url=settings.DATABASE_URL_psycopg,
    echo=False,
    pool_size=5,
    max_overflow=10,
)

# Настройка асинхронного двигателя
async_engine = create_async_engine(
    url=settings.DATABASE_URL_asyncpg,
    echo=False,
    pool_size=5,
    max_overflow=10,
)

In [3]:
async def get_async():
    # Использование подключения
    async with async_engine.connect() as conn:
        result = await conn.execute(text("SELECT VERSION()"))  # Используем text() для строки SQL
        # for row in result:
        #     print(f"Database version: {row[0]}")

        print(f'{result.first()=}')

In [9]:
await get_async()

result.first()=('PostgreSQL 17.2 (Debian 17.2-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)
